# Librairies

In [2]:
import portfolio_functions as pf

# Portfolio Analysis


## Critères ESG

Nous allons créer notre portefeuille d'actions en se basant sur les critères ESG. Comme vu en cours cela s'appelle un ESG filter :
* exclude x% of firms with the worts ESG score
* keep firms with the best ESG momentum
* take a specific KPI

## Volatilites

Nous calculons la volatilites des actions avec la fonction asset_vol.

## Analyse des returns

Prendre uniquement les assets avec des returns anualisés positifs.